<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum Basis Statistiek / SQL </h1>

<h2>Imports<h2>

In [1]:
import os
os.environ['export PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] ='notebook'
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

In [2]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
from PIL import Image
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.pandas as ps
import sqlite3

<h2>Extract, Transform and Load</h2>

<h3>Extract</h3>

In [3]:
# https://raw.githubusercontent.com/ajturner/acetate/master/places/Countries-Europe.csv
eurCsvFile = "../csv/countries_europe.csv"
euCsvFile = "../csv/european_union.csv"

 # Create Pandas DataFrames
eurDF = pd.read_csv(eurCsvFile)
euDF = pd.read_csv(euCsvFile)
# print(europeDF)
# print(euDF)

<h3>Transform</h3>

<h3>Exercise 1</h3>
<h4>Gegeven</h4>

In [4]:
benelux = {"Belgium", "The Netherlands", "Luxembourg"}
nordics = {"Denmark", "Finland", "Iceland", "Norway"}
baltics = {"Estonia", "Latvia", "Lithuania"}

<h4>Gevraagd</h4>
<ol>
    <li>Voeg de kolom <u>group</u> toe aan de dataset <u>eurDF</u> met daarin de groep waartoe een land behoort volgens bovenstaande sets</li>
    <li>Ga na of er nog meer landengroepen zijn binnen Europa en voeg die ook toe</li>
    <li>Sla deze verrijkte dataset op in een nieuw CSV bestand</li>
</ol>

<h4>Oplossing</h4>

In [10]:
eurDF['group']=None

In [51]:
# Oplossing

#print(eurDF.keys())
eurDF.name =='Belgium'.index(True)
#regionstr in ['benelux','nordics','baltics']:
#    regmem= eval(a) 
    

#eurDF

TypeError: must be str, not bool

<h2>SQLite</h2>

<h3>Load</h3>

<p>Maak een lege database</p>

In [ ]:
!sqlite3 ../db/europe.db "VACUUM;"

In [ ]:
# Connect to SQLite database
dbConnection = sqlite3.connect('../db/europe.db')

<p>Maak tabellen o.b.v. de dataset</p>

In [ ]:
eurDF.to_sql('countries', dbConnection, if_exists='replace', index = False)
euDF.to_sql('eu', dbConnection, if_exists='replace', index = False)

In [ ]:
dbCursor = dbConnection.cursor()

query = """
    SELECT *
    FROM countries
    INNER JOIN eu
    ON countries.Code = eu.Code
    LIMIT 10
"""

result = dbCursor.execute(query).fetchall()
print(result)
print(type(result))

In [ ]:
countriesQuery = """
    SELECT *
    FROM countries
    LIMIT 10
"""

In [ ]:
result = dbCursor.execute(countriesQuery).fetchall()
print(result)

<h2>Afhankelijke en onafhankelijke variabelen</h2>

<h3>Overlappende uitkomsten</h3>

In [ ]:
eu = {"Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "The Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"}

In [ ]:
venn3([benelux, baltics, eu], ('Benelux', 'Baltics', 'EU'))

<h3>Exercise 2</h3>
<p>
    Gebruik SQL queries voor het verkrijgen van
<ol>
    <li>P(Nordics); de kans dat een land onderdeel is van de Nordics</li>
    <li><u>P(EU)</u>; de kans dat een Europees land lid is van de Europese Unie</li>
    <li><u>P(EU and Nordics)</u>; de kans dat een land onderdeel is van de Norics en lid is van de EU</li>
</ol>
</p>

In [ ]:
# Oplossing

In [ ]:
# Close connection to SQLite database
dbConnection.close()